In [ ]:
# 필요한 라이브러리 설치
!pip install -q chromadb sentence-transformers langchain langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.2 MB/s eta 0:00:00
 

In [ ]:
!pip install fastapi uvicorn
!pip install transformers
!pip install accelerate
!pip install huggingface_hub
!pip install langchain-google-genai
!pip install google-generativeai
!pip install html2text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 46.7 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.17 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.4 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.17
    Uninstalling google-ai-generativelanguage-0.6.17:
      Successfully uninstalled google-ai-generativelanguage-0.6.17
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.3 requires google-ai-generativelanguage<0.7.0,>=0.6.16, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [ ]:
!pip install -U langchain langchain_experimental
!pip install rank-bm25

In [ ]:
import os
import json
import requests
from google.colab import drive, userdata

In [ ]:
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

In [ ]:
# Google Drive 마운트
drive.mount('/content/drive')

# ChromaDB 저장 경로 설정
CHROMA_DB_PATH = "/content/drive/MyDrive/chromadb_notion"
os.makedirs(CHROMA_DB_PATH, exist_ok=True)

Mounted at /content/drive


In [ ]:
# Notion API 인증 설정
NOTION_TOKEN = userdata.get('NOTION_TOKEN')  # Colab에서 안전하게 토큰 관리
NOTION_VERSION = "2022-06-28"  # API 버전

headers = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Notion-Version": NOTION_VERSION,
    "Content-Type": "application/json"
}

def get_page_content(page_id):
    """
    페이지 ID를 기반으로 페이지 내용을 가져옵니다.

    Args:
        page_id (str): Notion 페이지 ID (대시 제외)

    Returns:
        dict: 페이지 속성 및 내용
    """
    # 페이지 속성 가져오기
    page_url = f"https://api.notion.com/v1/pages/{page_id}"
    response = requests.get(page_url, headers=headers)

    if response.status_code != 200:
        print(f"오류 발생: {response.status_code}")
        print(response.text)
        return None

    page_data = response.json()

    # 페이지 내용(블록) 가져오기
    blocks_url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    response = requests.get(blocks_url, headers=headers)

    if response.status_code != 200:
        print(f"블록 가져오기 오류: {response.status_code}")
        print(response.text)
        return page_data

    blocks_data = response.json()

    # 페이지 데이터에 블록 내용 추가
    page_data["content"] = blocks_data["results"]

    return page_data

def fetch_all_blocks(block_id):
    """
    블록의 모든 하위 블록을 재귀적으로 가져옵니다.

    Args:
        block_id (str): 블록 ID

    Returns:
        list: 모든 하위 블록 목록
    """
    blocks_url = f"https://api.notion.com/v1/blocks/{block_id}/children"
    response = requests.get(blocks_url, headers=headers)

    if response.status_code != 200:
        print(f"블록 가져오기 오류: {response.status_code}")
        print(response.text)
        return []

    blocks_data = response.json()["results"]

    # 하위 블록이 있는 블록 타입들
    has_children_types = [
        "paragraph", "bulleted_list_item", "numbered_list_item",
        "toggle", "child_page", "child_database", "column_list",
        "column", "table", "synced_block"
    ]

    all_blocks = []

    for block in blocks_data:
        all_blocks.append(block)

        # 하위 블록이 있는 경우 재귀적으로 가져오기
        if block.get("has_children") and block.get("type") in has_children_types:
            children = fetch_all_blocks(block["id"])
            # 하위 블록이 있는 경우에만 children 키 추가
            if children:
                block["children"] = children

    return all_blocks

def extract_text_content(blocks):
    """
    블록에서 텍스트 내용만 추출합니다.

    Args:
        blocks (list): 블록 목록

    Returns:
        str: 추출된 텍스트 내용
    """
    text_content = ""

    for block in blocks:
        block_type = block.get("type")

        if block_type == "paragraph":
            rich_text = block.get("paragraph", {}).get("rich_text", [])
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_1":
            rich_text = block.get("heading_1", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "# " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_2":
            rich_text = block.get("heading_2", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "## " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_3":
            rich_text = block.get("heading_3", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "### " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "bulleted_list_item":
            rich_text = block.get("bulleted_list_item", {}).get("rich_text", [])
            text_content += "• "
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "numbered_list_item":
            rich_text = block.get("numbered_list_item", {}).get("rich_text", [])
            # 실제로는 번호가 순차적으로 증가해야 하지만, 여기서는 간단하게 표시
            text_content += "1. "
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "to_do":
            rich_text = block.get("to_do", {}).get("rich_text", [])
            checked = block.get("to_do", {}).get("checked", False)
            text_content += "[{}] ".format("x" if checked else " ")
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "code":
            rich_text = block.get("code", {}).get("rich_text", [])
            language = block.get("code", {}).get("language", "")
            text_content += f"```{language}\n"
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n```\n\n"

        # 하위 블록이 있는 경우 재귀적으로 처리
        if "children" in block:
            text_content += extract_text_content(block["children"])

    return text_content



In [ ]:
def setup_chromadb():
    """
    ChromaDB 클라이언트와 컬렉션을 설정합니다.

    Returns:
        Collection: ChromaDB 컬렉션 객체
    """
    # 한국어 지원 임베딩 모델 설정
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
        model_name="jhgan/ko-sbert-sts"  # 한국어 지원 모델
    )

    # ChromaDB 클라이언트 설정
    client = chromadb.PersistentClient(
        path=CHROMA_DB_PATH,
        settings=Settings(anonymized_telemetry=False)
    )

    # 컬렉션 생성 또는 가져오기
    try:
        collection = client.get_collection(
            name="notion_pages",
            embedding_function=sentence_transformer_ef
        )
        print("기존 컬렉션을 로드했습니다.")
    except:
        collection = client.create_collection(
            name="notion_pages",
            embedding_function=sentence_transformer_ef,
            metadata={"description": "Notion 페이지 내용"}
        )
        print("새 컬렉션을 생성했습니다.")

    return collection

def semantic_chunking(text):
    """
    SemanticChunker를 사용하여 텍스트를 의미 단위로 분할합니다.

    Args:
        text (str): 청크로 나눌 텍스트

    Returns:
        list: 텍스트 청크 목록
    """
    # 한국어 지원 임베딩 모델 설정
    embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")

    # SemanticChunker 설정
    text_splitter = SemanticChunker(embedding_model)

    # 텍스트 분할
    chunks = text_splitter.split_text(text)

    return chunks

def store_page_in_chromadb(collection, page_id, page_title, text_content):
    """
    페이지 내용을 ChromaDB에 저장합니다.

    Args:
        collection: ChromaDB 컬렉션
        page_id (str): 페이지 ID
        page_title (str): 페이지 제목
        text_content (str): 페이지 텍스트 내용
    """
    # 시맨틱 청킹으로 텍스트를 청크로 나누기
    chunks = semantic_chunking(text_content)

    # 각 청크에 대한 ID, 메타데이터 생성
    ids = []
    documents = []
    metadatas = []

    for i, chunk in enumerate(chunks):
        chunk_id = f"{page_id}-chunk-{i}"
        ids.append(chunk_id)
        documents.append(chunk)
        metadatas.append({
            "page_id": page_id,
            "page_title": page_title,
            "chunk_index": i,
            "total_chunks": len(chunks),
            "source": "notion"
        })

    # ChromaDB에 저장
    collection.add(
        ids=ids,
        documents=documents,
        metadatas=metadatas
    )

    print(f"'{page_title}' 페이지를 {len(chunks)}개 청크로 ChromaDB에 저장했습니다.")

# 메인 실행 코드
def main():
    # 사용자로부터 페이지 ID 입력받기
    page_id = input("Notion 페이지 ID를 입력하세요 (예: 053b5fdc70c34fe8b931c06e320c3083): ")
    page_id = page_id.replace("-", "")  # 대시가 있으면 제거

    print("페이지 내용을 가져오는 중...")

    # 페이지 내용 가져오기
    page_data = get_page_content(page_id)

    if not page_data:
        print("페이지를 가져올 수 없습니다.")
        return

    # 모든 블록 가져오기 (재귀적으로)
    print("모든 블록을 재귀적으로 가져오는 중...")
    all_blocks = fetch_all_blocks(page_id)

    # 텍스트 내용 추출
    text_content = extract_text_content(all_blocks)

    # 페이지 제목 가져오기
    page_title = "무제"
    title_property = page_data.get("properties", {}).get("title", {}).get("title", [])
    if title_property:
        page_title = " ".join([text.get("plain_text", "") for text in title_property])

    print(f"\n=== 페이지 기본 정보 ===")
    print(f"제목: {page_title}")
    print(f"내용 길이: {len(text_content)} 자")

    # ChromaDB 설정
    print("\nChromaDB 설정 중...")
    collection = setup_chromadb()

    # ChromaDB에 저장
    print("\n페이지 내용을 ChromaDB에 저장 중...")
    store_page_in_chromadb(collection, page_id, page_title, text_content)

    print(f"\nChromaDB에 성공적으로 저장되었습니다.")
    print(f"저장 경로: {CHROMA_DB_PATH}")
    print(f"컬렉션 이름: notion_pages")

    # ChromaDB 쿼리 예제
    print("\n저장된 내용 검색 예제:")
    query = input("검색할 키워드를 입력하세요: ")
    results = collection.query(
        query_texts=[query],
        n_results=2
    )

    if results["documents"][0]:
        print("\n검색 결과:")
        for i, (doc, metadata) in enumerate(zip(results["documents"][0], results["metadatas"][0])):
            print(f"\n결과 {i+1}:")
            print(f"페이지: {metadata['page_title']} (청크 {metadata['chunk_index']+1}/{metadata['total_chunks']})")
            print(f"내용 일부: {doc[:200]}...")
    else:
        print("검색 결과가 없습니다.")

In [ ]:
if __name__ == "__main__":
    main()

Notion 페이지 ID를 입력하세요 (예: 053b5fdc70c34fe8b931c06e320c3083): 1dd124ddd313806d840be6d8aa46680f
페이지 내용을 가져오는 중...
모든 블록을 재귀적으로 가져오는 중...

=== 페이지 기본 정보 ===
제목: 어쩐지 미술에서 뇌과학이 보인다
내용 길이: 2149 자

ChromaDB 설정 중...
기존 컬렉션을 로드했습니다.

페이지 내용을 ChromaDB에 저장 중...
'어쩐지 미술에서 뇌과학이 보인다' 페이지를 3개 청크로 ChromaDB에 저장했습니다.

ChromaDB에 성공적으로 저장되었습니다.
저장 경로: /content/drive/MyDrive/chromadb_notion
컬렉션 이름: notion_pages

저장된 내용 검색 예제:
검색할 키워드를 입력하세요: 추상미술

검색 결과:

결과 1:
페이지: 어쩐지 미술에서 뇌과학이 보인다 (청크 3/3)
내용 일부: 크게 두방향입니다. 데 쿠닝(De Kooning)과 폴락(Pollock)의 행동미술(gesturial painting, action painting), 로스코(Rothko)와 모리스 루이스(Morris Louis) 등의 색면화(color field painting)입니다. 더 나아가면 빛을 가지고 정서를 자아내는 댄 플래빈(Dan Flavin)과 제임스 터...

결과 2:
페이지: 어쩐지 미술에서 뇌과학이 보인다 (청크 1/3)
내용 일부: 1️⃣ 한줄 평

이과에게 가장 완벽한 추상화 설명

♓ Inuit Points ★★★☆☆

뇌과학 이론에 기반한 추상미술의 원리입니다. 뇌는 시각 정보를 상향식으로 인식하는 한편, 다시 그 정보를 하향식으로 해석합니다. 추상화는 이 하향 정보 처리 과정을 적극적으로 활용합니다. 다만 미학적 뇌(aesthetic brain) 연구의 초기 단계의 책이라서인지...


In [ ]:
from rank_bm25 import BM25Okapi
import numpy as np

In [ ]:
def create_hybrid_retriever(collection, alpha=0.5):
    """
    BM25와 의미적 유사도를 결합한 하이브리드 검색 함수를 생성합니다.

    Args:
        collection: ChromaDB 컬렉션
        alpha (float): BM25와 의미적 유사도 간의 가중치 (0에 가까울수록 BM25 중심, 1에 가까울수록 의미적 유사도 중심)

    Returns:
        function: 검색 쿼리를 처리하는 하이브리드 검색 함수
    """
    # 모든 문서 가져오기
    print("컬렉션에서 모든 문서 가져오는 중...")
    results = collection.get()

    all_docs = results["documents"]
    all_ids = results["ids"]
    all_metadatas = results["metadatas"]

    # BM25 모델 초기화
    print("BM25 인덱스 생성 중...")
    # 문서를 토큰화 (간단한 방식으로)
    tokenized_docs = [doc.split() for doc in all_docs]
    bm25 = BM25Okapi(tokenized_docs)

    def retrieve(query, top_k=5):
        """
        주어진 쿼리에 대해 BM25와 의미적 유사도를 결합하여 검색합니다.

        Args:
            query (str): 검색 쿼리
            top_k (int): 반환할 상위 결과 수

        Returns:
            list: 검색 결과 목록 (문서, 메타데이터, 점수)
        """
        print(f"'{query}' 검색 중...")

        # BM25 점수 계산
        tokenized_query = query.split()
        bm25_scores = np.array(bm25.get_scores(tokenized_query))

        # 정규화 (0-1 사이로)
        if bm25_scores.max() > 0:
            bm25_scores = bm25_scores / bm25_scores.max()

        # 의미적 유사도 검색
        semantic_results = collection.query(
            query_texts=[query],
            n_results=len(all_docs)
        )

        semantic_scores = np.array(semantic_results["distances"][0])
        # 거리를 유사도로 변환 (1 - 거리) 및 정규화
        semantic_scores = 1 - semantic_scores

        # 하이브리드 점수 계산 (가중 평균)
        combined_scores = (1 - alpha) * bm25_scores + alpha * semantic_scores

        # 상위 결과 선택
        top_indices = np.argsort(combined_scores)[-top_k:][::-1]

        # 결과 구성
        results = []
        for idx in top_indices:
            results.append({
                "document": all_docs[idx],
                "metadata": all_metadatas[idx],
                "score": combined_scores[idx],
                "bm25_score": bm25_scores[idx],
                "semantic_score": semantic_scores[idx]
            })

        return results

    return retrieve

# 사용 예:
def test_hybrid_retrieval():
    # ChromaDB 설정
    print("ChromaDB 설정 중...")
    collection = setup_chromadb()

    # 하이브리드 검색 함수 생성 (BM25와 의미적 유사도 균형)
    retriever = create_hybrid_retriever(collection, alpha=0.5)

    # 검색 실행
    query = input("검색할 키워드를 입력하세요: ")
    results = retriever(query, top_k=3)

    # 결과 출력
    print("\n검색 결과:")
    for i, result in enumerate(results):
        print(f"\n결과 {i+1} (종합 점수: {result['score']:.4f}):")
        print(f"BM25 점수: {result['bm25_score']:.4f}, 의미적 유사도: {result['semantic_score']:.4f}")
        print(f"페이지: {result['metadata']['page_title']} (청크 {result['metadata']['chunk_index']+1}/{result['metadata']['total_chunks']})")
        print(f"내용 일부: {result['document'][:200]}...")

ChromaDB 설정 중...
기존 컬렉션을 로드했습니다.
컬렉션에서 모든 문서 가져오는 중...
BM25 인덱스 생성 중...
검색할 키워드를 입력하세요: 스파이크
'스파이크' 검색 중...

검색 결과:

결과 1 (종합 점수: -242.3772):
BM25 점수: 0.0000, 의미적 유사도: -484.7545
페이지:  넥스트 커머스 (청크 1/3)
내용 일부: 1️⃣ 한줄 평

쿠팡이 잘 나가고, 티메프가 왜 망했는지 알만하다

♓ Inuit Points ★★★☆☆

전자상거래는 떠오르는 산업이었지만, 이젠 성숙산업이 되어 버렸지요. 하지만 그 물밑은 살벌한 전쟁터입니다. 책은 결국 대형 플랫폼 위주로 편제될 수 밖에 없는 커머스 산업의 구조를 살피고, 거기에 대응하는 흐름을 짚어봅니다. 이 시대 커머스 산업을 ...

결과 2 (종합 점수: -256.0772):
BM25 점수: 0.0000, 의미적 유사도: -512.1544
페이지:  넥스트 커머스 (청크 2/3)
내용 일부: 별 셋 주었습니다. ❤️  To whom it matters

• 유통, 제조, 자영업에 종사하시는 분
• 자꾸 테무 광고가 뜨시는 분
🎢 Stories Related

• 큰 차원의 틀은 있지만, 13인 공저라 주장이 정교하거나 설득력 넘치진 않습니다. • 편집에 공을 들여 챕터간 중복이 거의 없고 각자 관점을 잘 녹여 낸 점은 좋습니다. • 어쩔수 없이...

결과 3 (종합 점수: -262.9131):
BM25 점수: 0.0000, 의미적 유사도: -525.8263
페이지:  넥스트 커머스 (청크 3/3)
내용 일부: 그나마 둘러볼 지점으로는 다음을 꼽습니다. • 롱테일 다양화
• 커뮤니티 형 커머스
• 뉴 커머스(게임화, 공동구매)
• 라이브 커머스
• 오프라인 활용 (유통이 아닌 미디어 포지션으로 접근)
• 라이프스타일 제안
가장 최저가를 제공하겠다는 아마존의 약속은 아직도 지켜지고 있을겁니다. 하지만 우리는 매트릭스 안으로 들어가 행복한 꿈을 꾸는 것 아닌가 싶습니

In [ ]:
if __name__ == "__main__":
    test_hybrid_retrieval()

ChromaDB 설정 중...
기존 컬렉션을 로드했습니다.
컬렉션에서 모든 문서 가져오는 중...
BM25 인덱스 생성 중...
검색할 키워드를 입력하세요: 삼체의 영문판 번역가
'삼체의 영문판 번역가' 검색 중...

검색 결과:

결과 1 (종합 점수: -211.4846):
BM25 점수: 0.0000, 의미적 유사도: -422.9692
페이지:  넥스트 커머스 (청크 1/3)
내용 일부: 1️⃣ 한줄 평

쿠팡이 잘 나가고, 티메프가 왜 망했는지 알만하다

♓ Inuit Points ★★★☆☆

전자상거래는 떠오르는 산업이었지만, 이젠 성숙산업이 되어 버렸지요. 하지만 그 물밑은 살벌한 전쟁터입니다. 책은 결국 대형 플랫폼 위주로 편제될 수 밖에 없는 커머스 산업의 구조를 살피고, 거기에 대응하는 흐름을 짚어봅니다. 이 시대 커머스 산업을 ...

결과 2 (종합 점수: -224.0483):
BM25 점수: 0.0000, 의미적 유사도: -448.0966
페이지:  넥스트 커머스 (청크 2/3)
내용 일부: 별 셋 주었습니다. ❤️  To whom it matters

• 유통, 제조, 자영업에 종사하시는 분
• 자꾸 테무 광고가 뜨시는 분
🎢 Stories Related

• 큰 차원의 틀은 있지만, 13인 공저라 주장이 정교하거나 설득력 넘치진 않습니다. • 편집에 공을 들여 챕터간 중복이 거의 없고 각자 관점을 잘 녹여 낸 점은 좋습니다. • 어쩔수 없이...

결과 3 (종합 점수: -225.6470):
BM25 점수: 0.0000, 의미적 유사도: -451.2941
페이지:  넥스트 커머스 (청크 3/3)
내용 일부: 그나마 둘러볼 지점으로는 다음을 꼽습니다. • 롱테일 다양화
• 커뮤니티 형 커머스
• 뉴 커머스(게임화, 공동구매)
• 라이브 커머스
• 오프라인 활용 (유통이 아닌 미디어 포지션으로 접근)
• 라이프스타일 제안
가장 최저가를 제공하겠다는 아마존의 약속은 아직도 지켜지고 있을겁니다. 하지만 우리는 매트릭스 안으로 들어가 행복한 꿈